<a href="https://colab.research.google.com/github/AdityaPandeyCN/ONNX_Operator_Implementation/blob/main/Reluoperator_CUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Download the pre-built ROOT tarball from GitHub Releases
!wget -q --show-progress https://github.com/MohamedElashri/ROOT/releases/download/ubuntu/root_v6.30.04_Ubuntu_Python3.11.zip
# Step 2: Extract the ROOT files
!unzip -q root_v6.30.04_Ubuntu_Python3.11.zip

# Step 3: Install missing system dependencies for ROOT
!sudo ldconfig & apt-get install -y git dpkg-dev cmake g++ gcc binutils libx11-dev libxpm-dev libxft-dev libxext-dev tar gfortran subversion libpython3.11-dev

# Step 4: Remove the tarball to free up space
!rm -f root_v6.30.04_Ubuntu_Python3.11.zip

# Step 5: Install Compatible libssl

!wget http://archive.ubuntu.com/ubuntu/pool/main/o/openssl/libssl1.1_1.1.1f-1ubuntu2_amd64.deb
!sudo dpkg -i libssl1.1_1.1.1f-1ubuntu2_amd64.deb
!rm -f libssl1.1_1.1.1f-1ubuntu2_amd64.deb


root_v6.30.04_Ubunt 100%[===================>] 272.11M  7.32MB/s    in 4.6s    
/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libumf.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libur_adapter_level_zero.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libur_adapter_opencl.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libhwloc.so.15 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libur_loader.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtcm_debug.so.1 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0

In [ ]:
import sys
import ctypes

# Step 1: Append ROOT paths to Python
sys.path.append("root_build/")
sys.path.append("root_build/bin/")
sys.path.append("root_build/include/")
sys.path.append("root_build/lib/")

# Step 2: Load the required shared libraries (.so files)
ctypes.cdll.LoadLibrary("root_build/lib/libCore.so")
ctypes.cdll.LoadLibrary("root_build/lib/libThread.so")
ctypes.cdll.LoadLibrary("root_build/lib/libTreePlayer.so")

print("ROOT Libraries Loaded Successfully!")


ROOT Libraries Loaded Successfully!


In [ ]:
!mkdir -p /content/tmva_cuda_project/{include/TMVA,src,test,build}

In [ ]:
%%writefile /content/tmva_cuda_project/include/TMVA/RTensor.hxx
#ifndef TMVA_SOFIE_RTENSOR
#define TMVA_SOFIE_RTENSOR

#include <vector>
#include <memory>
#include <stdexcept>
#include <algorithm>
#include <numeric>
#include <functional>
#include <iostream>

namespace TMVA {
namespace Experimental {
namespace SOFIE {

// Forward declaration
template <typename T> class RTensor;

// Simple tensor class for TMVA SOFIE
template <typename T>
class RTensor {
private:
    std::vector<size_t> fShape;
    std::shared_ptr<T[]> fData;
    size_t fSize;

public:
    // Default constructor
    RTensor() : fSize(0) {}

    // Constructor with shape
    RTensor(const std::vector<size_t>& shape) : fShape(shape) {
        fSize = std::accumulate(shape.begin(), shape.end(),
                              (size_t)1, std::multiplies<size_t>());
        fData = std::shared_ptr<T[]>(new T[fSize]());
    }

    // Constructor with shape and data
    RTensor(const std::vector<size_t>& shape, const T* data) : fShape(shape) {
        fSize = std::accumulate(shape.begin(), shape.end(),
                              (size_t)1, std::multiplies<size_t>());
        fData = std::shared_ptr<T[]>(new T[fSize]);
        std::copy(data, data + fSize, fData.get());
    }

    // Get shape
    const std::vector<size_t>& GetShape() const {
        return fShape;
    }

    // Get size
    size_t GetSize() const {
        return fSize;
    }

    // Get data
    T* GetData() {
        return fData.get();
    }

    const T* GetData() const {
        return fData.get();
    }

    // Access element
    T& operator[](size_t index) {
        if (index >= fSize) {
            throw std::out_of_range("RTensor index out of range");
        }
        return fData[index];
    }

    const T& operator[](size_t index) const {
        if (index >= fSize) {
            throw std::out_of_range("RTensor index out of range");
        }
        return fData[index];
    }
};

}}} // namespace TMVA::Experimental::SOFIE

#endif // TMVA_SOFIE_RTENSOR

Writing /content/tmva_cuda_project/include/TMVA/RTensor.hxx


In [ ]:
%%writefile /content/tmva_cuda_project/include/TMVA/SOFIE_common.hxx
#ifndef TMVA_SOFIE_COMMON
#define TMVA_SOFIE_COMMON

#include "TMVA/RTensor.hxx"
#include <string>
#include <vector>
#include <memory>
#include <stdexcept>
#include <iostream>
#include <unordered_map>

namespace TMVA {
namespace Experimental {
namespace SOFIE {

// Basic tensor type enum
enum class ETensorType { FLOAT, DOUBLE, INT64, BOOL };

// Dimension structure for dynamic shapes
struct Dim {
    std::string name;
    size_t size;
};

// Helper functions for shape conversion
inline size_t ConvertShapeToLength(const std::vector<size_t>& shape) {
    size_t length = 1;
    for (auto& dim : shape) {
        length *= dim;
    }
    return length;
}

// Get string representation of type
template<typename T>
std::string GetTensorTypeName() {
    if (std::is_same<T, float>::value) return "float";
    if (std::is_same<T, double>::value) return "double";
    if (std::is_same<T, int64_t>::value) return "int64_t";
    if (std::is_same<T, bool>::value) return "bool";
    return "unknown";
}

// Get ETensorType from C++ type
template<typename T>
ETensorType GetTemplatedType(T) {
    if (std::is_same<T, float>::value) return ETensorType::FLOAT;
    if (std::is_same<T, double>::value) return ETensorType::DOUBLE;
    if (std::is_same<T, int64_t>::value) return ETensorType::INT64;
    if (std::is_same<T, bool>::value) return ETensorType::BOOL;
    throw std::runtime_error("Unsupported type in GetTemplatedType");
}

// Simple tensor info structure
struct TensorInfo {
    ETensorType type;
    std::vector<size_t> shape;
};

// Structure for dynamic tensor info
struct DynamicTensorInfo {
    ETensorType type;
    std::vector<Dim> shape;
};

// Structure for input tensor info
struct InputTensorInfo {
    ETensorType type;
    std::vector<Dim> shape;
};

// Structure for initialized tensor
struct InitializedTensor {
    ETensorType type;
    std::vector<size_t> shape;
    std::shared_ptr<void> data;
    bool isConstant = false;
    bool isWritable = true;
};

// Options for code generation
enum class Options {
    kDefault = 0,
    kNoSession = 1
};

}}} // namespace TMVA::Experimental::SOFIE

#endif // TMVA_SOFIE_COMMON

Writing /content/tmva_cuda_project/include/TMVA/SOFIE_common.hxx


In [ ]:
%%writefile /content/tmva_cuda_project/include/TMVA/ROperator.hxx
#ifndef TMVA_SOFIE_ROPERATOR
#define TMVA_SOFIE_ROPERATOR

#include "TMVA/SOFIE_common.hxx"
#include <string>
#include <vector>

namespace TMVA {
namespace Experimental {
namespace SOFIE {

// Forward declaration
class RModel;

// Base class for all operators
class ROperator {
public:
    virtual ~ROperator() = default;
    virtual void Initialize(RModel& model) = 0;
    virtual std::string Generate(std::string OpName) = 0;

    // Common members
    std::vector<std::string> fInputTensorNames;
    std::vector<std::string> fOutputTensorNames;
};

}}} // namespace TMVA::Experimental::SOFIE

#endif // TMVA_SOFIE_ROPERATOR

Writing /content/tmva_cuda_project/include/TMVA/ROperator.hxx


In [ ]:
%%writefile /content/tmva_cuda_project/include/TMVA/RModel.hxx
#ifndef TMVA_SOFIE_RMODEL
#define TMVA_SOFIE_RMODEL

#include "TMVA/SOFIE_common.hxx"
#include <unordered_map>
#include <string>
#include <vector>
#include <memory>
#include <map>
#include <algorithm>

namespace TMVA {
namespace Experimental {
namespace SOFIE {

// Mock RModel class for our implementation
class RModel {
private:
    std::string fName;
    std::string fParsedDateTime;
    bool fIsInitialized = false;
    int fVerbose = 1;

    std::unordered_map<std::string, InputTensorInfo> fInputTensorInfos;
    std::unordered_map<std::string, TensorInfo> fReadyInputTensorInfos;
    std::unordered_map<std::string, InitializedTensor> fInitializedTensors;
    std::unordered_map<std::string, TensorInfo> fIntermediateTensorInfos;
    std::unordered_map<std::string, DynamicTensorInfo> fDynamicTensorInfos;
    std::vector<std::string> fOutputTensorNames;
    std::vector<std::string> fInputTensorNames;

public:
    RModel() = default;
    RModel(std::string name, std::string parsedtime) : fName(name), fParsedDateTime(parsedtime) {}

    int Verbose() const { return fVerbose; }

    const std::vector<size_t>& GetTensorShape(const std::string& name) {
        // First check intermediate tensors
        auto it = fIntermediateTensorInfos.find(name);
        if (it != fIntermediateTensorInfos.end()) {
            return it->second.shape;
        }

        // Check initialized tensors
        auto it2 = fInitializedTensors.find(name);
        if (it2 != fInitializedTensors.end()) {
            return it2->second.shape;
        }

        // Check input tensors
        auto it3 = fReadyInputTensorInfos.find(name);
        if (it3 != fReadyInputTensorInfos.end()) {
            return it3->second.shape;
        }

        throw std::runtime_error("Tensor not found: " + name);
    }

    const ETensorType& GetTensorType(const std::string& name) {
        // First check intermediate tensors
        auto it = fIntermediateTensorInfos.find(name);
        if (it != fIntermediateTensorInfos.end()) {
            return it->second.type;
        }

        // Check initialized tensors
        auto it2 = fInitializedTensors.find(name);
        if (it2 != fInitializedTensors.end()) {
            return it2->second.type;
        }

        // Check input tensors
        auto it3 = fReadyInputTensorInfos.find(name);
        if (it3 != fReadyInputTensorInfos.end()) {
            return it3->second.type;
        }

        throw std::runtime_error("Tensor type not found: " + name);
    }

    bool CheckIfTensorAlreadyExist(const std::string& name) {
        return (fIntermediateTensorInfos.find(name) != fIntermediateTensorInfos.end()) ||
               (fInitializedTensors.find(name) != fInitializedTensors.end()) ||
               (fReadyInputTensorInfos.find(name) != fReadyInputTensorInfos.end()) ||
               (fInputTensorInfos.find(name) != fInputTensorInfos.end());
    }

    // Add input tensor info with full shape
    void AddInputTensorInfo(const std::string& name, ETensorType type, const std::vector<size_t>& shape) {
        TensorInfo info;
        info.type = type;
        info.shape = shape;
        fReadyInputTensorInfos[name] = info;

        // Also add to input tensor names if not already there
        if (std::find(fInputTensorNames.begin(), fInputTensorNames.end(), name) == fInputTensorNames.end()) {
            fInputTensorNames.push_back(name);
        }
    }

    // Add intermediate tensor
    void AddIntermediateTensor(const std::string& name, ETensorType type, const std::vector<size_t>& shape) {
        TensorInfo info;
        info.type = type;
        info.shape = shape;
        fIntermediateTensorInfos[name] = info;
    }

    // Add output tensor names
    void AddOutputTensorNameList(const std::vector<std::string>& names) {
        fOutputTensorNames = names;
    }

    // Initialize model (simplified for mock)
    void Initialize(int batchSize = -1) {
        fIsInitialized = true;

        if (Verbose()) {
            std::cout << "Model initialized with batch size: " <<
                (batchSize == -1 ? "default" : std::to_string(batchSize)) << std::endl;
        }
    }
};

}}} // namespace TMVA::Experimental::SOFIE

#endif // TMVA_SOFIE_RMODEL

Writing /content/tmva_cuda_project/include/TMVA/RModel.hxx


In [ ]:
%%writefile /content/tmva_cuda_project/include/TMVA/ROperator_Relu_CUDA.hxx
#ifndef TMVA_SOFIE_ROPERATOR_RELU_CUDA
#define TMVA_SOFIE_ROPERATOR_RELU_CUDA

#include "TMVA/ROperator.hxx"
#include "TMVA/SOFIE_common.hxx"
#include <cuda_runtime.h>
#include <vector>
#include <string>

namespace TMVA {
namespace Experimental {
namespace SOFIE {

template <typename T>
class ROperator_Relu_CUDA final : public ROperator
{
private:
   std::string fNX;      // Input tensor name
   std::string fNY;      // Output tensor name
   std::vector<size_t> fShape;  // Tensor shape

public:
   ROperator_Relu_CUDA() = default;

   ROperator_Relu_CUDA(std::string nameX, std::string nameY):
      fNX(nameX), fNY(nameY) {
         fInputTensorNames = { nameX };
         fOutputTensorNames = { nameY };
      }

   // Type and shape inference
   std::vector<ETensorType> TypeInference(std::vector<ETensorType> input) {
      return input;  // ReLU preserves input type
   }

   std::vector<std::vector<size_t>> ShapeInference(std::vector<std::vector<size_t>> input) {
      return input;  // ReLU preserves input shape
   }

   // Required ROperator interface methods
   void Initialize(RModel& model) override;
   std::string Generate(std::string OpName) override;
};

// Declare template specializations
extern template class ROperator_Relu_CUDA<float>;
extern template class ROperator_Relu_CUDA<double>;
extern template class ROperator_Relu_CUDA<int64_t>;

}}} // namespace TMVA::Experimental::SOFIE

#endif // TMVA_SOFIE_ROPERATOR_RELU_CUDA

Writing /content/tmva_cuda_project/include/TMVA/ROperator_Relu_CUDA.hxx


In [ ]:
%%writefile /content/tmva_cuda_project/src/ROperator_Relu_CUDA.cu
#include "TMVA/ROperator_Relu_CUDA.hxx"
#include <sstream>

// CUDA kernel for ReLU operation
__global__ void reluKernelFloat(const float* input, float* output, size_t size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < size) {
        output[idx] = fmaxf(0.0f, input[idx]);
    }
}

// CUDA kernel for ReLU with double precision
__global__ void reluKernelDouble(const double* input, double* output, size_t size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < size) {
        output[idx] = fmax(0.0, input[idx]);
    }
}

// CUDA kernel for ReLU with int64
__global__ void reluKernelInt64(const int64_t* input, int64_t* output, size_t size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < size) {
        output[idx] = (input[idx] > 0) ? input[idx] : 0;
    }
}

namespace TMVA {
namespace Experimental {
namespace SOFIE {

template <typename T>
void ROperator_Relu_CUDA<T>::Initialize(RModel& model)
{
    if (!model.CheckIfTensorAlreadyExist(fNX)) {
        throw std::runtime_error("TMVA SOFIE Relu CUDA: Input tensor " + fNX + " not found in model");
    }

    // Get shape from the model
    fShape = model.GetTensorShape(fNX);

    // Add output tensor to the model with same type and shape as input
    model.AddIntermediateTensor(fNY, model.GetTensorType(fNX), fShape);

    if (model.Verbose()) {
        std::cout << "TMVA SOFIE Relu CUDA: " << fNX << " -> " << fNY << std::endl;
    }
}

template <typename T>
std::string ROperator_Relu_CUDA<T>::Generate(std::string OpName)
{
    if (fShape.empty()) {
        throw std::runtime_error("TMVA SOFIE Relu CUDA: Called Generate without initialization");
    }

    std::stringstream out;
    size_t length = 1;
    for (auto& dim : fShape) {
        length *= dim;
    }

    std::string typeName = GetTensorTypeName<T>();

    // Begin code generation
    out << "\n// " << OpName << " ReLU CUDA implementation\n";

    // 1. Define the kernel
    out << "__global__ void " << OpName << "_relu_kernel(const " << typeName << "* input, "
        << typeName << "* output, size_t size) {\n";
    out << "    int idx = blockIdx.x * blockDim.x + threadIdx.x;\n";
    out << "    if (idx < size) {\n";

    // Type-specific implementation
    if (std::is_same<T, float>::value) {
        out << "        output[idx] = fmaxf(0.0f, input[idx]);\n";
    } else if (std::is_same<T, double>::value) {
        out << "        output[idx] = fmax(0.0, input[idx]);\n";
    } else {
        out << "        output[idx] = (input[idx] > 0) ? input[idx] : 0;\n";
    }

    out << "    }\n";
    out << "}\n\n";

    // 2. Execution code block
    out << "{\n";  // Begin scope

    // Calculate launch configuration
    out << "    // Calculate execution configuration\n";
    out << "    size_t size = " << length << ";\n";
    out << "    int blockSize = 256;\n";
    out << "    int numBlocks = (size + blockSize - 1) / blockSize;\n\n";

    // GPU Memory allocation
    out << "    // Allocate device memory\n";
    out << "    " << typeName << "* d_input = nullptr;\n";
    out << "    " << typeName << "* d_output = nullptr;\n";
    out << "    cudaError_t cudaStatus;\n\n";

    // Error handling and memory management
    out << "    // CUDA memory allocation\n";
    out << "    cudaStatus = cudaMalloc(&d_input, size * sizeof(" << typeName << "));\n";
    out << "    if (cudaStatus != cudaSuccess) {\n";
    out << "        std::cerr << \"cudaMalloc failed for input: \" << cudaGetErrorString(cudaStatus) << std::endl;\n";
    out << "        goto " << OpName << "_cleanup;\n";
    out << "    }\n\n";

    out << "    cudaStatus = cudaMalloc(&d_output, size * sizeof(" << typeName << "));\n";
    out << "    if (cudaStatus != cudaSuccess) {\n";
    out << "        std::cerr << \"cudaMalloc failed for output: \" << cudaGetErrorString(cudaStatus) << std::endl;\n";
    out << "        goto " << OpName << "_cleanup;\n";
    out << "    }\n\n";

    // Copy input to device
    out << "    // Copy input to device\n";
    out << "    cudaStatus = cudaMemcpy(d_input, tensor_" << fNX << ", size * sizeof(" << typeName << "), cudaMemcpyHostToDevice);\n";
    out << "    if (cudaStatus != cudaSuccess) {\n";
    out << "        std::cerr << \"cudaMemcpy to device failed: \" << cudaGetErrorString(cudaStatus) << std::endl;\n";
    out << "        goto " << OpName << "_cleanup;\n";
    out << "    }\n\n";

    // Launch kernel
    out << "    // Launch kernel\n";
    out << "    " << OpName << "_relu_kernel<<<numBlocks, blockSize>>>(d_input, d_output, size);\n\n";

    // Check for kernel errors
    out << "    // Check for kernel errors\n";
    out << "    cudaStatus = cudaGetLastError();\n";
    out << "    if (cudaStatus != cudaSuccess) {\n";
    out << "        std::cerr << \"CUDA kernel launch failed: \" << cudaGetErrorString(cudaStatus) << std::endl;\n";
    out << "        goto " << OpName << "_cleanup;\n";
    out << "    }\n\n";

    // Synchronize
    out << "    // Wait for kernel completion\n";
    out << "    cudaStatus = cudaDeviceSynchronize();\n";
    out << "    if (cudaStatus != cudaSuccess) {\n";
    out << "        std::cerr << \"cudaDeviceSynchronize failed: \" << cudaGetErrorString(cudaStatus) << std::endl;\n";
    out << "        goto " << OpName << "_cleanup;\n";
    out << "    }\n\n";

    // Copy result back to host
    out << "    // Copy result back to host\n";
    out << "    cudaStatus = cudaMemcpy(tensor_" << fNY << ", d_output, size * sizeof(" << typeName << "), cudaMemcpyDeviceToHost);\n";
    out << "    if (cudaStatus != cudaSuccess) {\n";
    out << "        std::cerr << \"cudaMemcpy to host failed: \" << cudaGetErrorString(cudaStatus) << std::endl;\n";
    out << "        goto " << OpName << "_cleanup;\n";
    out << "    }\n\n";

    // Cleanup section
    out << OpName << "_cleanup:\n";
    out << "    // Clean up device memory\n";
    out << "    if (d_input) cudaFree(d_input);\n";
    out << "    if (d_output) cudaFree(d_output);\n\n";

    // CPU fallback if CUDA fails
    out << "    // CPU fallback if CUDA execution failed\n";
    out << "    if (cudaStatus != cudaSuccess) {\n";
    out << "        std::cerr << \"Using CPU fallback for ReLU operation\" << std::endl;\n";
    out << "        for (size_t i = 0; i < size; i++) {\n";
    out << "            tensor_" << fNY << "[i] = (tensor_" << fNX << "[i] > 0) ? tensor_" << fNX << "[i] : 0;\n";
    out << "        }\n";
    out << "    }\n";

    out << "}\n";  // End scope

    return out.str();
}

// Explicit template instantiations
template class ROperator_Relu_CUDA<float>;
template class ROperator_Relu_CUDA<double>;
template class ROperator_Relu_CUDA<int64_t>;

}}} // namespace TMVA::Experimental::SOFIE

Writing /content/tmva_cuda_project/src/ROperator_Relu_CUDA.cu


In [ ]:
%%writefile /content/tmva_cuda_project/test/test_relu_cuda.cu
#include "TMVA/ROperator_Relu_CUDA.hxx"
#include "TMVA/RModel.hxx"
#include <iostream>
#include <vector>
#include <chrono>
#include <iomanip>

using namespace TMVA::Experimental::SOFIE;

// Function to print tensor data
template <typename T>
void printTensor(const std::vector<T>& data, const std::vector<size_t>& shape) {
    if (shape.size() == 1) {
        for (size_t i = 0; i < std::min(data.size(), size_t(10)); i++) {
            std::cout << std::fixed << std::setprecision(2) << data[i] << " ";
        }
        if (data.size() > 10) std::cout << "...";
        std::cout << std::endl;
    } else if (shape.size() == 2) {
        for (size_t i = 0; i < std::min(shape[0], size_t(5)); i++) {
            for (size_t j = 0; j < std::min(shape[1], size_t(10)); j++) {
                std::cout << std::fixed << std::setprecision(2) << data[i * shape[1] + j] << " ";
            }
            if (shape[1] > 10) std::cout << "...";
            std::cout << std::endl;
        }
        if (shape[0] > 5) std::cout << "..." << std::endl;
    }
}

int main() {
    std::cout << "Testing TMVA SOFIE CUDA ReLU Operator" << std::endl;
    std::cout << "=====================================" << std::endl;

    try {
        // Create a model
        RModel model("cuda_relu_test", "2025-03-14");

        // Create input tensor shape and add to model
        std::vector<size_t> shape = {4, 4};
        model.AddInputTensorInfo("input", ETensorType::FLOAT, shape);

        // Initialize the model
        model.Initialize();

        // Create ReLU CUDA operator
        ROperator_Relu_CUDA<float> reluOp("input", "output");

        // Initialize operator
        reluOp.Initialize(model);

        // Generate code
        std::string generatedCode = reluOp.Generate("TestRelu");

        // Print code excerpt
        std::cout << "\nGenerated CUDA code (excerpt):" << std::endl;
        std::cout << "----------------------------" << std::endl;
        std::cout << generatedCode.substr(0, 300) << "...\n" << std::endl;

        std::cout << "\nReLU CUDA operator test completed successfully!" << std::endl;

        return 0;
    } catch (const std::exception& e) {
        std::cerr << "Error: " << e.what() << std::endl;
        return 1;
    }
}

Writing /content/tmva_cuda_project/test/test_relu_cuda.cu


In [ ]:
%%writefile /content/tmva_cuda_project/CMakeLists.txt
cmake_minimum_required(VERSION 3.10)
project(TMVA_SOFIE_CUDA CUDA CXX)

# Set C++ standard
set(CMAKE_CXX_STANDARD 14)
set(CMAKE_CUDA_STANDARD 14)
set(CMAKE_CUDA_ARCHITECTURES 70)

# Find CUDA
find_package(CUDA REQUIRED)

# Include directories
include_directories(
    ${CMAKE_CURRENT_SOURCE_DIR}/include
    ${CUDA_INCLUDE_DIRS}
)

# Add CUDA operator implementation
cuda_add_executable(test_relu_cuda
    test/test_relu_cuda.cu
    src/ROperator_Relu_CUDA.cu
)

# Link against CUDA libraries
target_link_libraries(test_relu_cuda
    ${CUDA_LIBRARIES}
)

Writing /content/tmva_cuda_project/CMakeLists.txt


In [ ]:
# Build the project
!cd /content/tmva_cuda_project && cmake -B build && cmake --build build

-- The CUDA compiler identification is NVIDIA 12.5.82 with host compiler GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting CUDA compiler ABI info
-- Detecting CUDA compiler ABI info - done
-- Check for working CUDA compiler: /usr/local/cuda/bin/nvcc - skipped
-- Detecting CUDA compile features
-- Detecting CUDA compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
CMake Warning (dev) at CMakeLists.txt:10 (find_package):
  Policy CMP0146 is not set: The FindCUDA module is removed.  Run "cmake
  --help-policy CMP0146" for policy details.  Use the cmake_policy command to
  set the policy and suppress this warning.

This warning is for project developers.  Use -Wno-dev to suppress it.

-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found T

In [ ]:
# First, let's update the ROperator_Relu_CUDA.cu file to include RModel.hxx explicitly
%%writefile /content/tmva_cuda_project/src/ROperator_Relu_CUDA.cu
#include "TMVA/ROperator_Relu_CUDA.hxx"
#include "TMVA/RModel.hxx"  // Explicitly include RModel.hxx
#include <sstream>

// CUDA kernel for ReLU operation
__global__ void reluKernelFloat(const float* input, float* output, size_t size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < size) {
        output[idx] = fmaxf(0.0f, input[idx]);
    }
}

// CUDA kernel for ReLU with double precision
__global__ void reluKernelDouble(const double* input, double* output, size_t size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < size) {
        output[idx] = fmax(0.0, input[idx]);
    }
}

// CUDA kernel for ReLU with int64
__global__ void reluKernelInt64(const int64_t* input, int64_t* output, size_t size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < size) {
        output[idx] = (input[idx] > 0) ? input[idx] : 0;
    }
}

namespace TMVA {
namespace Experimental {
namespace SOFIE {

template <typename T>
void ROperator_Relu_CUDA<T>::Initialize(RModel& model)
{
    if (!model.CheckIfTensorAlreadyExist(fNX)) {
        throw std::runtime_error("TMVA SOFIE Relu CUDA: Input tensor " + fNX + " not found in model");
    }

    // Get shape from the model
    fShape = model.GetTensorShape(fNX);

    // Add output tensor to the model with same type and shape as input
    model.AddIntermediateTensor(fNY, model.GetTensorType(fNX), fShape);

    if (model.Verbose()) {
        std::cout << "TMVA SOFIE Relu CUDA: " << fNX << " -> " << fNY << std::endl;
    }
}

template <typename T>
std::string ROperator_Relu_CUDA<T>::Generate(std::string OpName)
{
    if (fShape.empty()) {
        throw std::runtime_error("TMVA SOFIE Relu CUDA: Called Generate without initialization");
    }

    std::stringstream out;
    size_t length = 1;
    for (auto& dim : fShape) {
        length *= dim;
    }

    std::string typeName = GetTensorTypeName<T>();

    // Begin code generation
    out << "\n// " << OpName << " ReLU CUDA implementation\n";

    // 1. Define the kernel
    out << "__global__ void " << OpName << "_relu_kernel(const " << typeName << "* input, "
        << typeName << "* output, size_t size) {\n";
    out << "    int idx = blockIdx.x * blockDim.x + threadIdx.x;\n";
    out << "    if (idx < size) {\n";

    // Type-specific implementation
    if (std::is_same<T, float>::value) {
        out << "        output[idx] = fmaxf(0.0f, input[idx]);\n";
    } else if (std::is_same<T, double>::value) {
        out << "        output[idx] = fmax(0.0, input[idx]);\n";
    } else {
        out << "        output[idx] = (input[idx] > 0) ? input[idx] : 0;\n";
    }

    out << "    }\n";
    out << "}\n\n";

    // 2. Execution code block
    out << "{\n";  // Begin scope

    // Calculate launch configuration
    out << "    // Calculate execution configuration\n";
    out << "    size_t size = " << length << ";\n";
    out << "    int blockSize = 256;\n";
    out << "    int numBlocks = (size + blockSize - 1) / blockSize;\n\n";

    // GPU Memory allocation
    out << "    // Allocate device memory\n";
    out << "    " << typeName << "* d_input = nullptr;\n";
    out << "    " << typeName << "* d_output = nullptr;\n";
    out << "    cudaError_t cudaStatus;\n\n";

    // Error handling and memory management
    out << "    // CUDA memory allocation\n";
    out << "    cudaStatus = cudaMalloc(&d_input, size * sizeof(" << typeName << "));\n";
    out << "    if (cudaStatus != cudaSuccess) {\n";
    out << "        std::cerr << \"cudaMalloc failed for input: \" << cudaGetErrorString(cudaStatus) << std::endl;\n";
    out << "        goto " << OpName << "_cleanup;\n";
    out << "    }\n\n";

    out << "    cudaStatus = cudaMalloc(&d_output, size * sizeof(" << typeName << "));\n";
    out << "    if (cudaStatus != cudaSuccess) {\n";
    out << "        std::cerr << \"cudaMalloc failed for output: \" << cudaGetErrorString(cudaStatus) << std::endl;\n";
    out << "        goto " << OpName << "_cleanup;\n";
    out << "    }\n\n";

    // Copy input to device
    out << "    // Copy input to device\n";
    out << "    cudaStatus = cudaMemcpy(d_input, tensor_" << fNX << ", size * sizeof(" << typeName << "), cudaMemcpyHostToDevice);\n";
    out << "    if (cudaStatus != cudaSuccess) {\n";
    out << "        std::cerr << \"cudaMemcpy to device failed: \" << cudaGetErrorString(cudaStatus) << std::endl;\n";
    out << "        goto " << OpName << "_cleanup;\n";
    out << "    }\n\n";

    // Launch kernel
    out << "    // Launch kernel\n";
    out << "    " << OpName << "_relu_kernel<<<numBlocks, blockSize>>>(d_input, d_output, size);\n\n";

    // Check for kernel errors
    out << "    // Check for kernel errors\n";
    out << "    cudaStatus = cudaGetLastError();\n";
    out << "    if (cudaStatus != cudaSuccess) {\n";
    out << "        std::cerr << \"CUDA kernel launch failed: \" << cudaGetErrorString(cudaStatus) << std::endl;\n";
    out << "        goto " << OpName << "_cleanup;\n";
    out << "    }\n\n";

    // Synchronize
    out << "    // Wait for kernel completion\n";
    out << "    cudaStatus = cudaDeviceSynchronize();\n";
    out << "    if (cudaStatus != cudaSuccess) {\n";
    out << "        std::cerr << \"cudaDeviceSynchronize failed: \" << cudaGetErrorString(cudaStatus) << std::endl;\n";
    out << "        goto " << OpName << "_cleanup;\n";
    out << "    }\n\n";

    // Copy result back to host
    out << "    // Copy result back to host\n";
    out << "    cudaStatus = cudaMemcpy(tensor_" << fNY << ", d_output, size * sizeof(" << typeName << "), cudaMemcpyDeviceToHost);\n";
    out << "    if (cudaStatus != cudaSuccess) {\n";
    out << "        std::cerr << \"cudaMemcpy to host failed: \" << cudaGetErrorString(cudaStatus) << std::endl;\n";
    out << "        goto " << OpName << "_cleanup;\n";
    out << "    }\n\n";

    // Cleanup section
    out << OpName << "_cleanup:\n";
    out << "    // Clean up device memory\n";
    out << "    if (d_input) cudaFree(d_input);\n";
    out << "    if (d_output) cudaFree(d_output);\n\n";

    // CPU fallback if CUDA fails
    out << "    // CPU fallback if CUDA execution failed\n";
    out << "    if (cudaStatus != cudaSuccess) {\n";
    out << "        std::cerr << \"Using CPU fallback for ReLU operation\" << std::endl;\n";
    out << "        for (size_t i = 0; i < size; i++) {\n";
    out << "            tensor_" << fNY << "[i] = (tensor_" << fNX << "[i] > 0) ? tensor_" << fNX << "[i] : 0;\n";
    out << "        }\n";
    out << "    }\n";

    out << "}\n";  // End scope

    return out.str();
}

// Explicit template instantiations
template class ROperator_Relu_CUDA<float>;
template class ROperator_Relu_CUDA<double>;
template class ROperator_Relu_CUDA<int64_t>;

}}} // namespace TMVA::Experimental::SOFIE

Overwriting /content/tmva_cuda_project/src/ROperator_Relu_CUDA.cu


In [ ]:
%%writefile /content/tmva_cuda_project/include/TMVA/ROperator_Relu_CUDA.hxx
#ifndef TMVA_SOFIE_ROPERATOR_RELU_CUDA
#define TMVA_SOFIE_ROPERATOR_RELU_CUDA

#include "TMVA/RModel.hxx"  // Include RModel.hxx first
#include "TMVA/ROperator.hxx"
#include "TMVA/SOFIE_common.hxx"
#include <cuda_runtime.h>
#include <vector>
#include <string>

namespace TMVA {
namespace Experimental {
namespace SOFIE {

template <typename T>
class ROperator_Relu_CUDA final : public ROperator
{
private:
   std::string fNX;      // Input tensor name
   std::string fNY;      // Output tensor name
   std::vector<size_t> fShape;  // Tensor shape

public:
   ROperator_Relu_CUDA() = default;

   ROperator_Relu_CUDA(std::string nameX, std::string nameY):
      fNX(nameX), fNY(nameY) {
         fInputTensorNames = { nameX };
         fOutputTensorNames = { nameY };
      }

   // Type and shape inference
   std::vector<ETensorType> TypeInference(std::vector<ETensorType> input) {
      return input;  // ReLU preserves input type
   }

   std::vector<std::vector<size_t>> ShapeInference(std::vector<std::vector<size_t>> input) {
      return input;  // ReLU preserves input shape
   }

   // Required ROperator interface methods
   void Initialize(RModel& model) override;
   std::string Generate(std::string OpName) override;
};

// Declare template specializations
extern template class ROperator_Relu_CUDA<float>;
extern template class ROperator_Relu_CUDA<double>;
extern template class ROperator_Relu_CUDA<int64_t>;

}}} // namespace TMVA::Experimental::SOFIE

#endif // TMVA_SOFIE_ROPERATOR_RELU_CUDA

Overwriting /content/tmva_cuda_project/include/TMVA/ROperator_Relu_CUDA.hxx


In [ ]:
# Clean and rebuild the project
!cd /content/tmva_cuda_project && rm -rf build && mkdir -p build && cd build && cmake .. && make

-- The CUDA compiler identification is NVIDIA 12.5.82 with host compiler GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting CUDA compiler ABI info
-- Detecting CUDA compiler ABI info - done
-- Check for working CUDA compiler: /usr/local/cuda/bin/nvcc - skipped
-- Detecting CUDA compile features
-- Detecting CUDA compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
CMake Warning (dev) at CMakeLists.txt:10 (find_package):
  Policy CMP0146 is not set: The FindCUDA module is removed.  Run "cmake
  --help-policy CMP0146" for policy details.  Use the cmake_policy command to
  set the policy and suppress this warning.

This warning is for project developers.  Use -Wno-dev to suppress it.

-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found T

In [ ]:
# Run the test
!cd /content/tmva_cuda_project/build && ./test_relu_cuda

Testing TMVA SOFIE CUDA ReLU Operator
Model initialized with batch size: default
TMVA SOFIE Relu CUDA: input -> output

Generated CUDA code (excerpt):
----------------------------

// TestRelu ReLU CUDA implementation
__global__ void TestRelu_relu_kernel(const float* input, float* output, size_t size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < size) {
        output[idx] = fmaxf(0.0f, input[idx]);
    }
}

{
    // Calculate execution configuration
 ...


ReLU CUDA operator test completed successfully!
